In [1]:
import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import math
import random


class IntelliGoal(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(s):
        super(IntelliGoal, s).__init__()
        
        s.screen_width = 1240
        s.screen_height = 800
        s.border_width = 10
        s.screen = None
        
        s.goal_radius = 20
        s.player_radius = 10
        
        s.player_speed = 1
        
        
        # State information
        s.player_x = 0
        s.player_y = 0
        s.goal_x = 0.0
        s.goal_y = 0.0
        s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
        s.prev_distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)
        s.reward = 0
        
        s.done = False
        s.trucated = False
        s.truncation_step_limit = 1000
        s.truncation_step_counter = 0
        s.goal_spawn_range = 300
        
        s.prev_goal_position = (0, 0)  # Previous position of the goal
        s.position_change_timer = 0  # Timer to track position changes of the goal

        s.goal_speed = 0.3
        s.goal_direction = (0, 0)  # Initial direction of the goal
        s.direction_change_timer = 0
        s.direction_change_interval = 100  # Interval for changing direction (in frames)
        s.constant_goal_speed_increment = 0.000
        s.level_up_goal_speed_increment = 0.0
        
        
        # State and Action spaces
        s.action_space = spaces.Discrete(5) # 0: No movement, 1: Up, 2: Down, 3: Left, 4: Right
        # observation space = [player_x, player_y, goal_x, goal_y]
        s.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 0]), 
            high=np.array([s.screen_width, s.screen_height, s.screen_width, s.screen_height]), 
            dtype=np.int32
        )
    
    def seed(s, seed=None):
        if seed is not None and (seed < 0 or seed >= 2**32):
            raise ValueError("Seed must be between 0 and 2**32 - 1")
        s.np_random, seed = gym.utils.seeding.np_random(seed)
        return [seed]
        
    def reset(s, seed=0):
        s.seed(seed)
        
        s.player_x = s.screen_width // 2
        s.player_y = s.screen_height // 2
        
        s.goal_x, s.goal_y = s.respawn_goal()
        
        s.reward = 0
        s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
        
        
        s.done = False
        s.trucated = False
        s.truncation_step_counter = 0
        
        return s.state, {}
    
    def respawn_goal(s):
        s.goal_x = s.player_x + random.randint(-s.goal_spawn_range, s.goal_spawn_range)
        s.goal_y = s.player_y + random.randint(-s.goal_spawn_range, s.goal_spawn_range)
        
        # clip the goal to the screen
        offset = s.border_width + s.goal_radius
        s.goal_x = min( max(s.goal_x, offset), s.screen_width - offset)
        s.goal_y = min( max(s.goal_y, offset), s.screen_height - offset)
        
        # pygame.time.wait(300)
        
        s.prev_distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)
        
        return s.goal_x, s.goal_y
        
    def step(s, action):
        #################
        #do more here?
        #################
        
        s.reward = 0
        s.do_action(action)
        s.calculate_step_reward()

        s.dynamic_goal()
        
        s.truncation_step_counter += 1
        if s.truncation_step_counter >= s.truncation_step_limit:
            s.reward -= 1
            s.trucated = True
            s.done = True

        if s.reward < -1:
            # s.reward = -1
            s.trucated = True
        
        # if s.reward > 1:
        #     s.reward = 1
        
        
        return s.state, s.reward, s.done, s.trucated, {}
        # return s.state, s.reward, False, False, {}
        
   
    def do_action(s, action):
        if not s.action_space.contains(action):
            print("Invalid Action")
        
        s.prev_distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)
        
        s.player_x += (action == 4)*s.player_speed - (action == 3)*s.player_speed 
        s.player_y += (action == 2)*s.player_speed - (action == 1)*s.player_speed
        
    def calculate_step_reward(s):
        # Distance between player and goal
        distance = math.sqrt((s.player_x - s.goal_x) ** 2 + (s.player_y - s.goal_y) ** 2)

        # Check if the red dot touches the border
        if s.player_x not in range(s.border_width, s.screen_width - s.border_width) or s.player_y not in range(s.border_width, s.screen_height - s.border_width):
            s.player_x, s.player_y = s.screen_width // 2, s.screen_height // 2
            s.reward = -1
            s.trucated = True
            s.done = True

        # Check if the goal is reached within a certain range
        if distance < s.goal_radius + s.player_radius:
            s.goal_x, s.goal_y = s.respawn_goal()
            s.reward += 1.5
            s.done = True  
    
        s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
        
        # Reward the agent for getting closer to the goal
        if s.prev_distance > distance:
            s.reward = 0.5
        if s.prev_distance < distance:
            s.reward = -0.7
        
        # Penalize the agent for taking too long
        # s.reward -= 0.00008
     
    def dynamic_goal(s):
            # Update goal position and direction
            if s.direction_change_timer <= 0:
                # Change direction
                s.goal_direction = (random.randint(-2, 2), random.randint(-2, 2))
                # Reset direction change timer
                s.direction_change_timer = s.direction_change_interval
            else:
                # Continue in the current direction
                s.goal_x += s.goal_direction[0] * s.goal_speed
                s.goal_y += s.goal_direction[1] * s.goal_speed
                s.direction_change_timer -= 1

            # Ensure the goal stays within the screen bounds with a margin inside the borders
            goal_margin = 25  # Adjust this value as needed
            s.goal_x = max(s.border_width + goal_margin, min(s.goal_x, s.screen_width - s.border_width - goal_margin))
            s.goal_y = max(s.border_width + goal_margin, min(s.goal_y, s.screen_height - s.border_width - goal_margin))

            # Increase goal speed gradually
            s.goal_speed += s.constant_goal_speed_increment

            # Check if the goal position is changing
            current_goal_position = (s.goal_x, s.goal_y)
            if current_goal_position == s.prev_goal_position:
                # Increment the timer if the position remains the same
                s.position_change_timer += 1
            else:
                # Reset the timer if the position changes
                s.position_change_timer = 0
                # Update the previous position
                s.prev_goal_position = current_goal_position

            # Check if the goal is stuck and change direction if necessary
            if s.position_change_timer >= 3:  #threshold as needed
                s.goal_direction = (random.randint(-2, 2), random.randint(-2, 2))
                s.position_change_timer = 0
            
            # Update the state
            s.state = np.array([s.player_x, s.player_y, s.goal_x, s.goal_y])
            
    def keyboard_input(s):
        keys = pygame.key.get_pressed()
        # 1: Up, 2: Down, 3: Left, 4: Right
        if keys[pygame.K_UP]:
            return 1
        if keys[pygame.K_DOWN]:
            return 2
        if keys[pygame.K_LEFT]:
            return 3
        if keys[pygame.K_RIGHT]:
            return 4
        return 0
        
    
    def render(s, mode = 'human'):
        if s.screen == None:
            pygame.init()
            s.screen = pygame.display.set_mode((s.screen_width, s.screen_height))
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
        s.player_x, s.player_y, s.goal_x, s.goal_y = s.state
        
        s.screen.fill((0, 0, 0))
        RED = (255, 0, 0)
        BLUE = (0, 0, 255)
        BORDER_COLOR = (255, 255, 0)
        
        # Draw the border
        pygame.draw.rect(s.screen, BORDER_COLOR, (0, 0, s.screen_width, s.border_width))
        pygame.draw.rect(s.screen, BORDER_COLOR, (0, 0, s.border_width, s.screen_height))
        pygame.draw.rect(s.screen, BORDER_COLOR, (0, s.screen_height - s.border_width, s.screen_width, s.border_width))
        pygame.draw.rect(s.screen, BORDER_COLOR, (s.screen_width - s.border_width, 0, s.border_width, s.screen_height))

        # Draw the red dot
        pygame.draw.circle(s.screen, RED, (s.player_x, s.player_y), 10)

        # Draw the blue dot
        pygame.draw.circle(s.screen, BLUE, (s.goal_x, s.goal_y), s.goal_radius)
        
        #Display Reward on pygame screen
        font = pygame.font.Font(None, 36)
        text = font.render("Reward: " + str(round(s.reward, 5)), True, (255, 255, 255))
        s.screen.blit(text, (10, 10))
        
        pygame.display.update()
    
    def close(s):
        pygame.quit()
        
        

pygame 2.5.2 (SDL 2.28.3, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Test Env with keyboard input actions
test_env = IntelliGoal()
test_env.reset(0)
done = False
i = 1

# while True:
#     try:
#         test_env.render()
#         action = test_env.keyboard_input()
#         state, reward, done, truncated, _i = test_env.step(action)
#         if done or truncated:
#             print("Reward at step - ", i, " : ", round(reward, 5))
#             i += 1
#             test_env.reset(0)
#     except:
#         print("Environment Closed")
#         break

# while True:
#     test_env.render()
#     action = test_env.keyboard_input()
#     state, reward, done, truncated, _i = test_env.step(action)
#     if done or truncated:
#         print("Reward at step - ", i, " : ", round(reward, 5))
#         i += 1
#         test_env.reset(0)   


In [3]:

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

In [4]:
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
# save_path = "./models/dqn_intelligoal_a1_works-100"
# save_path = "models\dqn_step-rewards_a2-good-stuff.zip"

save_path = "./models/dqn_step-rewards_a3-best_static_goals"
device = "cuda"

In [5]:
def make_env():
    def _init():
        env = IntelliGoal()
        env.render_mode = 'human'
        return env
    return _init

# Wrap your environment in a DummyVecEnv for compatibility
env = DummyVecEnv([make_env()])

model = DQN.load(save_path, env=env, device=device)

def run_model():
    obs = env.reset()  # This will now be in the correct batch format
    done = False
    while True:
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, done, info = env.step(action)
        env.render()
        if done:
            print("Episode Finished with Reward - ", rewards[0])
            obs = env.reset()

try:
    run_model()
except:
    print("Environment Closed")


Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Episode Finished with Reward -  0.5
Environment Closed
